In [1]:
"""vectorised faddeeva using masking instead of branching

All the if-then-else branches have been turned into masks, this avoids branching, and cache is  also presedved.

code is fully vectorised, and indexing is not used, hence turning it into parallel 
pattern using numba is very easy

this masking pattern was taken from the book "Structured Parallel Programming: Patterns for Efficient Computation" by James Reinders, on page 104 CHAPTER 3 Patterns: section: 3.6.3 Speculative Selection"""


import numpy as np
import numba

@numba.njit
def vec_faddeeva(z):
    """
    The Faddeeva function. Code adapted from
    https://github.com/tiagopereira/Transparency.jl/blob/966fb46c21d5847c28f6f2eaa5ea6ff569e25bf2/src/voigt.jl#L13.

    Parameters
    ----------
    z : complex

    Returns
    -------
    w : complex
    """

    INV_SQRT_PI = 1.0 /  np.sqrt(np.pi) 
    x = np.real(z)
    y = np.imag(z)

    zz = z * z

    s = np.abs(x) + y
    t = y - 1j * x
    
# region I
    mask1 = np.greater(s, 15.0) 
    region1 = 1j * INV_SQRT_PI * z / (zz -0.5) 

# region II
    mask2 = np.greater(s, 5.5) 
    region2 = (
            1j
            * (z * (zz * INV_SQRT_PI - 1.4104739589))
            / (0.75 + zz * (zz - 3.0))
        )

# region III
    mask3 = np.greater_equal(y, 0.195 * np.abs(x) - 0.176) 
    region3 = (
                16.4955
                + t * (20.20933 + t * (11.96482 + t * (3.778987 + 0.5642236 * t)))
            ) / (
                16.4955
                + t * (38.82363 + t * (39.27121 + t * (21.69274 + t * (6.699398 + t))))
            )

 # region IV
    mask4 = np.logical_not(mask3)
    u = t * t
    numerator = t * (
                36183.31
                - u
                * (
                    3321.99
                    - u
                    * (
                        1540.787
                        - u * (219.031 - u * (35.7668 - u * (1.320522 - u * 0.56419)))
                    )
                )
            )
    denominantor = 32066.6 - u * (
                24322.8
                - u
                * (
                    9022.23
                    - u * (2186.18 - u * (364.219 - u * (61.5704 - u * (1.84144 - u))))
                )
            )

    region4 = np.exp(u) - numerator / denominantor


    w = np.empty_like(z)

    w[mask1] = region1[mask1]
    w[mask2] = region2[mask2]
    w[mask3] = region3[mask3]
    w[mask4] = region4[mask4]

    return w


In [2]:

import numpy as np
import numba

@numba.njit(parallel=True)
def parallel_faddeeva(z):
    """
    The Faddeeva function. Code adapted from
    https://github.com/tiagopereira/Transparency.jl/blob/966fb46c21d5847c28f6f2eaa5ea6ff569e25bf2/src/voigt.jl#L13.

    Parameters
    ----------
    z : complex

    Returns
    -------
    w : complex
    """

    INV_SQRT_PI = 1.0 /  np.sqrt(np.pi) 
    x = np.real(z)
    y = np.imag(z)

    zz = z * z

    s = np.abs(x) + y
    t = y - 1j * x
    
# region I
    mask1 = np.greater(s, 15.0) 
    region1 = 1j * INV_SQRT_PI * z / (zz -0.5) 

# region II
    mask2 = np.greater(s, 5.5) 
    region2 = (
            1j
            * (z * (zz * INV_SQRT_PI - 1.4104739589))
            / (0.75 + zz * (zz - 3.0))
        )

# region III
    mask3 = np.greater_equal(y, 0.195 * np.abs(x) - 0.176) 
    region3 = (
                16.4955
                + t * (20.20933 + t * (11.96482 + t * (3.778987 + 0.5642236 * t)))
            ) / (
                16.4955
                + t * (38.82363 + t * (39.27121 + t * (21.69274 + t * (6.699398 + t))))
            )

 # region IV
    mask4 = np.logical_not(mask3)
    u = t * t
    numerator = t * (
                36183.31
                - u
                * (
                    3321.99
                    - u
                    * (
                        1540.787
                        - u * (219.031 - u * (35.7668 - u * (1.320522 - u * 0.56419)))
                    )
                )
            )
    denominantor = 32066.6 - u * (
                24322.8
                - u
                * (
                    9022.23
                    - u * (2186.18 - u * (364.219 - u * (61.5704 - u * (1.84144 - u))))
                )
            )

    region4 = np.exp(u) - numerator / denominantor


    w = np.empty_like(z)

    w[mask1] = region1[mask1]
    w[mask2] = region2[mask2]
    w[mask3] = region3[mask3]
    w[mask4] = region4[mask4]

    return w


In [3]:
# %load /mnt/9d11ac2c-0f5d-4794-bfe1-e81a350afec1/home/safa/Documents/gsoc_repos/stardis_smith7/stardis/opacities/voigt.py

#current implementation of Faddeva function

import numpy as np
import numba

@numba.njit
def faddeeva(z):
    """
    The Faddeeva function. Code adapted from
    https://github.com/tiagopereira/Transparency.jl/blob/966fb46c21d5847c28f6f2eaa5ea6ff569e25bf2/src/voigt.jl#L13.

    Parameters
    ----------
    z : complex

    Returns
    -------
    w : complex
    """
    s = abs(np.real(z)) + np.imag(z)

    if s > 15.0:
        # region I
        w = 1j * 1 / np.sqrt(np.pi) * z / (z**2 - 0.5)

    elif s > 5.5:
        # region II
        w = (
            1j
            * (z * (z**2 * 1 / np.sqrt(np.pi) - 1.4104739589))
            / (0.75 + z**2 * (z**2 - 3.0))
        )

    else:
        x = np.real(z)
        y = np.imag(z)
        t = y - 1j * x

        if y >= 0.195 * abs(x) - 0.176:
            # region III
            w = (
                16.4955
                + t * (20.20933 + t * (11.96482 + t * (3.778987 + 0.5642236 * t)))
            ) / (
                16.4955
                + t * (38.82363 + t * (39.27121 + t * (21.69274 + t * (6.699398 + t))))
            )

        else:
            # region IV
            u = t * t
            numerator = t * (
                36183.31
                - u
                * (
                    3321.99
                    - u
                    * (
                        1540.787
                        - u * (219.031 - u * (35.7668 - u * (1.320522 - u * 0.56419)))
                    )
                )
            )
            denominantor = 32066.6 - u * (
                24322.8
                - u
                * (
                    9022.23
                    - u * (2186.18 - u * (364.219 - u * (61.5704 - u * (1.84144 - u))))
                )
            )
            w = np.exp(u) - numerator / denominantor

    return w

@numba.njit
def voigt_profile(delta_nu, doppler_width, gamma):
    """
    Calculates the Voigt profile, the convolution of a Lorentz profile
    and a Gaussian profile.

    Parameters
    ----------
    delta_nu : float
        Difference between the frequency that the profile is being evaluated at
        and the line's resonance frequency.
    doppler_width : float
        Doppler width for Gaussian profile.
    gamma : float
        Broadening parameter for Lorentz profile.

    Returns
    -------
    phi : float
        Value of Voigt profile.
    """
    z = (delta_nu + (gamma / (4 * np.pi)) * 1j) / doppler_width
    phi = np.real(faddeeva(z)) / (np.sqrt(np.pi) * doppler_width)
    return phi


In [4]:
# testing setup

bins = 100000
x = np.linspace(-3, 3, bins)
y = np.linspace(-2, 2, bins)

z = x + 1j * y


In [5]:
# vectorizing current faddeeva function
vf = np.vectorize(faddeeva)
vf(z)


# testing both the vectoirised and parallel implementation against current implementation

np.testing.assert_allclose(vf(z), vec_faddeeva(z))
np.testing.assert_allclose(vf(z), parallel_faddeeva(z))

In [6]:
# Timing current faddeeva function

%timeit vf(z)

25.9 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
# Timing vectorised faddeeva function

%timeit vec_faddeeva(z)

16.7 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
# Timing Parallel faddeeva function

%timeit parallel_faddeeva(z)

6.5 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
# comparison with scipy version

from scipy.special import wofz

%timeit wofz(z)

14.1 ms ± 62.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
